In [16]:

import numpy as np 
import math
import time
from math import exp, log, pi, sqrt
from scipy import stats

In [48]:
class OptionsPricing:

    def __init__(self, S, K, T, r, q, sigma):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.q = q 
        self.sigma = sigma 

#================================ CALLS ====================================================================

    def call_price(self):

        d1 = (np.log(S/K) + t * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(t))
        d2 = d1 - sigma * sqrt(t)

        price = S * np.exp(-q * t) * stats.norm.cdf(d1) - K * np.exp(-r * t) * stats.norm.cdf(d2)

        return price

    def call_delta(self):

        d1 = (np.log(S/K) + t * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(t))
        d2 = d1 - sigma * sqrt(t)

        delta = stats.norm.cdf(d1) * exp(-q * T)

        return delta

    def call_gamma(self):

        d1 = (np.log(S/K) + t * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(t))
        d2 = d1 - sigma * sqrt(t)

        gamma = (exp(-1* d1 ** 2) / 2) / sqrt(2 * pi)

        return gamma


#================================ PUTS ====================================================================

    def put_price(self):

        d1 = (np.log(S/K) + t * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(t))
        d2 = d1 - sigma * sqrt(t)

        price = K * np.exp(-r * t) * stats.norm.cdf(-d2) - S * np.exp(-q * t) * stats.norm.cdf(-d1)

        return price

    def put_delta(self):

        d1 = (np.log(S/K) + t * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(t))
        d2 = d1 - sigma * sqrt(t)

        delta = (stats.norm.cdf(d1)-1) * exp(-q * T)

        return delta

    def put_gamma(self):

        d1 = (np.log(S/K) + t * (r - q + sigma ** 2 / 2)) / (sigma * sqrt(t))
        d2 = d1 - sigma * sqrt(t)

        gamma = (exp(-1* d1 ** 2) / 2) / sqrt(2 * pi)

        return gamma


In [49]:
S = 110
K = 100
r = 0.10
q = 0.08
T = 6/12
sigma = 0.25

In [50]:
model = OptionsPricing(S, K, T, r, q, sigma)

callprice = model.call_price()
calldelta = model.call_delta()
callgamma = model.call_gamma()
putprice = model.put_price()
putdelta = model.put_delta()
putgamma = model.call_gamma()





0.12491823438631462